# Import Nessesory Libries

In [ ]:
# import all the nessecery libries for machine learning project
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
import seaborn as sns
import matplotlib.pyplot as plt

# Import Data Sets

In [ ]:
# Import data set
outlet_data=pd.read_csv('Data Storn - Semi final data sets/outlets_data.csv')
freezer_data=pd.read_csv('Data Storn - Semi final data sets/freezer_data.csv')
product_data=pd.read_csv('Data Storn - Semi final data sets/product_data.csv')
sales_data=pd.read_csv('Data Storn - Semi final data sets/sales_data.csv')
week_data=pd.read_csv('Data Storn - Semi final data sets/week_data.csv')



In [ ]:
outlet_data

In [ ]:
# create a data frame for each outlate and product
outlet_data = outlet_data.groupby(['Outlet_ID'])['area(sqft)'].mean().reset_index()

In [ ]:
outlet_data

In [ ]:
# Check for duplicates in column 'A'
duplicates = outlet_data[outlet_data.duplicated(['Outlet_ID'], keep=False)]
print(duplicates)

In [ ]:
# get count of null values in each column
null_counts = outlet_data.isnull().sum()
# print the counts
print(null_counts)

In [ ]:
product_data

In [ ]:
sales_data

In [ ]:
# get count of null values in each column
null_counts = sales_data.isnull().sum()
# print the counts
print(null_counts)

In [ ]:
week_data

In [ ]:
# get count of null values in each column
null_counts = week_data.isnull().sum()
# print the counts
print(null_counts)

# Creating Dtaframes

#### create a data frame for each outlate and product

In [ ]:
# create a data frame for each outlate and product
sales_grouped_outlats = sales_data.groupby(['Outlet_ID', 'pid'])['no_units'].sum().reset_index()


In [ ]:
sales_grouped_outlats

In [ ]:
# pivot the dataframe to make item_description values as columns
pivoted_df_sales = sales_grouped_outlats.pivot(index='Outlet_ID', columns='pid', values='no_units')

# fill NaN values with 0
pivoted_df_sales.fillna(0, inplace=True)

# resetting the index
pivoted_df_sales = pivoted_df_sales.reset_index()

In [ ]:
pivoted_df_sales

In [ ]:
# add columns for each product to the outlet data frame

sales_grouped_outlats=(pd.merge(outlet_data, pivoted_df_sales, on='Outlet_ID', how='left'))

In [ ]:
sales_grouped_outlats

#### creating fetures for total item solds for bulk and impuls item

In [ ]:
sales_grouped_outlats['tot_bulk_itm'] = sales_grouped_outlats[['BP1', 'BP2', 'BP3', 'BP4', 'BP5']].sum(axis=1)
sales_grouped_outlats['tot_impulse_itm'] = sales_grouped_outlats[['IP1', 'IP2', 'IP3', 'IP4', 'IP5']].sum(axis=1)

In [ ]:
sales_grouped_outlats

#### Creatng feature for Total Icecream Volume sold

In [ ]:
sales_grouped_outlats['tot_icecream_vol']=(sales_grouped_outlats['tot_bulk_itm']*1.0)+(sales_grouped_outlats['tot_impulse_itm']*0.2)

In [ ]:
sales_grouped_outlats[['area(sqft)','BP1', 'BP2', 'BP3', 'BP4', 'BP5','IP1', 'IP2', 'IP3', 'IP4', 'IP5','tot_bulk_itm','tot_impulse_itm']] = sales_grouped_outlats[['area(sqft)','BP1', 'BP2', 'BP3', 'BP4', 'BP5','IP1', 'IP2', 'IP3', 'IP4', 'IP5','tot_bulk_itm','tot_impulse_itm']].astype(int)

In [ ]:
sales_grouped_outlats

#### Create a feature for total revenew

In [ ]:
product_data

In [ ]:
sales_grouped_outlats['tot_revnew']=((sales_grouped_outlats['BP1']*1200)+(sales_grouped_outlats['BP2']*1100)+(sales_grouped_outlats['BP3']*1150)+(sales_grouped_outlats['BP4']*1000)+(sales_grouped_outlats['BP5']*1100)+(sales_grouped_outlats['IP1']*100)+(sales_grouped_outlats['IP2']*90)+(sales_grouped_outlats['IP3']*110)+(sales_grouped_outlats['IP4']*100)+(sales_grouped_outlats['IP5']*100)).astype(float)

In [ ]:
sales_grouped_outlats

### Encode

In [ ]:
sales_grouped_outlats['Outlet_ID'] = sales_grouped_outlats['Outlet_ID'].str.replace(r'^ID', '').astype(int)

In [ ]:
sales_grouped_outlats

In [ ]:
sales_grouped_outlats.describe()

In [ ]:
# Print list of features
print(sales_grouped_outlats.columns.tolist())

# model to remove duploicate Outlet_ID

In [ ]:
# Group the DataFrame by column 'Outlet_ID' and get the groups with duplicates
groups = sales_grouped_outlats[['Outlet_ID','area(sqft)','tot_revnew']].groupby(['Outlet_ID']).filter(lambda x: len(x) > 1)

df_of_duplicates=pd.DataFrame()

# Iterate over the groups and print the duplicate rows
for name, group in groups.groupby(['Outlet_ID']):
    df_of_duplicates=df_of_duplicates.append(group)

In [ ]:
df_of_duplicates

# Visualization

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt


x = sales_grouped_outlats['area(sqft)']
y = sales_grouped_outlats['tot_revnew']

plt.scatter(x, y)

plt.show()

##### Corelation matrix

In [ ]:
# Create correlation matrix
corr = sales_grouped_outlats.corr()

# Set figure size
plt.figure(figsize=(12, 8))

# Plot correlation matrix as heatmap
sns.heatmap(corr, annot=True, cmap='coolwarm')

# Display plot
plt.show()

# Training the Model

In [ ]:
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans

# Load data from csv file into pandas DataFrame
data = sales_grouped_outlats

features = ['area(sqft)', 'BP1', 'BP2', 'BP3', 'BP4', 'BP5', 'IP1', 'IP2', 'IP3', 'IP4', 'IP5', 'tot_bulk_itm', 'tot_impulse_itm', 'tot_icecream_vol', 'tot_revnew']


n_clusters=10
iterations=500

# Select features for k-means clustering
X = data[features]

# Initialize and fit k-means clustering model
kmeans = KMeans(n_clusters=10,init='k-means++', n_init=2000, max_iter=500,tol=0.000001 ) # set number of clusters
kmeans.fit(X)

# Get predicted cluster labels and centroids
labels = kmeans.predict(X)




In [ ]:
# Create scatter plot with feature_1 on x-axis and feature_2 on y-axis

featur_1 = 'BP5'
featur_2 = 'tot_revnew'

fig, ax = plt.subplots(figsize=(10, 6))
ax.scatter(data[featur_1], data[featur_2], c=labels, s=50, cmap='viridis')

# Add legend with cluster labels
scatter = ax.scatter([],[], c=[], cmap='viridis')
plt.colorbar(scatter, label='Cluster')

# Set x and y axis labels
ax.set_xlabel(featur_1)
ax.set_ylabel(featur_2)

# Show plot
plt.show()

In [ ]:
# Create a DataFrame from the array of cluster lables
clusters_df = pd.DataFrame(labels, columns=['Clusters'])

In [ ]:
clusters_df

In [ ]:
outlats_with_clusters = pd.concat([sales_grouped_outlats, clusters_df], axis=1)

In [ ]:
outlats_with_clusters

In [ ]:
df_for_optimization=outlats_with_clusters[['Outlet_ID','area(sqft)','tot_revnew','tot_icecream_vol','Clusters']]

In [ ]:
df_for_optimization

In [ ]:
# create a csv file for the optimization data frame
df_for_optimization.to_csv('df_for_optimization.csv', index=False)